In [10]:
# %cd ..
!nvidia-smi

Sun Apr  5 09:07:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [11]:
!pip install gdown
!pip install tensorflow-gpu==1.15
!pip install --upgrade keras
 

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [12]:
%cd /content/drive/My\ Drive/person_detection/keras-retinanet
!pip install .
!python setup.py build_ext --inplace
%cd ../../../../sample_data/
!git clone https://github.com/Yannick947/WiderPerson.git
%cd ../../

/content/drive/My Drive/person_detection/keras-retinanet
Processing /content/drive/My Drive/person_detection/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=170573 sha256=682db35aa32a762ef7769b3a58fdb1e5efee858e0ec16ec90b0ba9e0cdfa1038
  Stored in directory: /root/.cache/pip/wheels/ca/78/63/58275b4a0f024168bf718212448cbba87f45aa1948fdc1a291
Successfully built keras-retinanet
  Found existing installation: keras-retinanet 0.5.1
    Uninstalling keras-retinanet-0.5.1:
      Successfully uninstalled keras-retinanet-0.5.1


running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.6/keras_retinanet/utils/compute_overlap.cpython-36m-x86_64-linux-gnu.so -> keras_retinanet/utils
/content/sample_data
fatal: destination path 'WiderPerson' already exists and is not an empty directory.
/


In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from IPython.core.debugger import set_trace
import shutil

import pdb

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
# tf.random.set_seed(RANDOM_SEED)


In [0]:
annot_path = '/content/drive/My Drive/person_detection/WiderPerson/Annotations'
images_path = '/content/drive/My Drive/person_detection/WiderPerson/Images'
annot_csv_path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations_new.csv'
keras_path = '/content/drive/My Drive/person_detection/keras-retinanet'
#Remove classes which shall be left out in the csv
classes_ids = {1:'pedestrian',
               2:'riders',
               3:'partially-visible',
               5:'crowd'}

# Training



In [0]:
#TODO: Make it also work for bigger architecture:"/fizyr/keras-retinanet/releases/download/0.5.1/resnet152_oid_v1.0.0.h5"

In [0]:
PRETRAINED_150_VANILLA = './snapshots/resnet152_oid_v1_0_0_h5'
PRETRAINED_MODEL_VANILLA = './snapshots/_pretrained_model.h5'
PRETRAINED_MODEL_WI = '../custom_models/resnet50_e15_b8_s1000_train_0304.h5'
PRETRAINED_MODEL_WI2 = './snapshots_secondtrainer/resnet50_csv_20.h5'
# URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
# urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL_VANILLA)


In [21]:
%cd /content/drive/My Drive/person_detection/keras-retinanet
!python keras_retinanet/bin/train.py  --backbone='resnet50' \
                                      --image-min-side=100 \
                                      --image-max-side=1300 \
                                      --weights {PRETRAINED_MODEL_VANILLA} \
                                      --tensorboard-dir 'tensorboard' \
                                      --lr 1e-5 \
                                      --batch-size 8 \
                                      --steps 1000 \
                                      --epochs 20 \
                                      --no-snapshots\
                                      --random-transfrom\
                                      csv annot_train_filtered_600_3000.csv classes.csv \
                                    # --val-annotations annot_test.csv \
                                    # --freeze-backbone \
                                    # --compute-val-loss \
                                    # --random-transform \

%cd ../../../../../

/content/drive/My Drive/person_detection/keras-retinanet
Using TensorFlow backend.
usage: train.py [-h]
                [--snapshot SNAPSHOT | --imagenet-weights | --weights WEIGHTS | --no-weights]
                [--backbone BACKBONE] [--batch-size BATCH_SIZE] [--gpu GPU]
                [--multi-gpu MULTI_GPU] [--multi-gpu-force]
                [--initial-epoch INITIAL_EPOCH] [--epochs EPOCHS]
                [--steps STEPS] [--lr LR] [--snapshot-path SNAPSHOT_PATH]
                [--tensorboard-dir TENSORBOARD_DIR] [--no-snapshots]
                [--no-evaluation] [--freeze-backbone] [--random-transform]
                [--image-min-side IMAGE_MIN_SIDE]
                [--image-max-side IMAGE_MAX_SIDE] [--no-resize]
                [--config CONFIG] [--weighted-average] [--compute-val-loss]
                [--multiprocessing] [--workers WORKERS]
                [--max-queue-size MAX_QUEUE_SIZE]
                {coco,pascal,kitti,oid,csv} ...
train.py: error: unrecognized argument

### Anchor Optimization

In [0]:
%cd /content/drive/My Drive/person_detection
!git clone https://github.com/martinzlocha/anchor-optimization.git
%cd anchor-optimization
!pip install .
!python setup.py build_ext --inplace

In [0]:
!python ./../anchor-optimization/optimize_anchors.py annot_train.csv


## Functions


In [0]:
# Generate the classes csv file
with open(annotations, newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for filename in os.listdir(images_path)[0:10]:
    if str(filename + '.txt') in annot:
      f = open(annot_path + '/' +  filename + '.txt', 'r')
      
      for index, line in enumerate(f): 
        if index == 0: 
          if line.strip() == '0':
            print('Not any  object in the image!')
          continue
          
        else: 
          split_line = line[:line.find('/')].split(' ')
          first_char = split_line.pop(0)
          split_line.insert(len(split_line), first_char)
          split_line.insert(0, images_path + '/' + filename)
          #convert from index to class label 
          try:
            split_line[-1] = classes_ids[int(split_line[-1])]
          except: 
            continue
          split_line[0] = split_line[0]

          csv_writer.writerow(split_line)
      
      f.close()

#create indexing csv file
with open('classes.csv', newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for index, key in enumerate(classes_ids.keys()):
    csv_writer.writerow([str(classes_ids[key]), index])


#Check for wrongly annotaded bounding boxes
path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations.csv'
def check_bb(path):
  colnames = ['filename', 'x1', 'y1', 'x2', 'y2', 'class_label']
  df = pd.read_csv(path, names=colnames)
  df_new = df.loc[(df.x1 < df.x2) & (df.y1 < df.y2), : ]
  print ('Reduces shape from ', df.shape, 'to ', df_new.shape)
  return df_new





In [0]:
def replace_annoation_folder(new_folder, annot_path, old_img_path):
  '''Start this function from folder root, otherwise wont work properly'''

  df_annot = pd.read_csv(annot_path + '/annotations_pathkeras.csv',
                         names=['name', 'x1', 'y1', 'x2', 'y2', 'label'])
  df_annot = df_annot.reset_index().drop(0).drop(columns='index')
  df_annot['name'] = df_annot['name'].str.replace(old_img_path, new_folder)
  df_annot = df_annot.dropna()
  df_annot.to_csv(annot_path + '/annotations_new.csv',
                  index=None, header=None)
  return


old_folder = '/content/drive/My Drive/PersonDetection/WiderPerson/Images'
new_folder = '/content/drive/My Drive/person_detection/WiderPerson/Images'
annot_path_csv = '/content/drive/My Drive/person_detection/keras-retinanet'
replace_annoation_folder(new_folder, annot_path_csv, old_folder)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [0]:
#Splitting into train and test data

train_df, test_df = train_test_split(
  pd.read_csv(annot_csv_path, header=None), 
  test_size=0.15, 
  random_state=RANDOM_SEED
)

train_df.to_csv(keras_path + '/annot_train.csv', header=None, index=None)
test_df.to_csv(keras_path + '/annot_test.csv', header=None, index=None)

/content/drive/My Drive/person_detection/WiderPerson/Annotations
